In [3]:
from redcap import Project
import pandas as pd
import datetime

#REDCap API configuration
api_url = 'https://ruff.mrc.gm:8443/redcap/api/'
api_key = 'D6807C091B1FC14F57CC9B2972D0B398'
project = Project(api_url, api_key)


In [4]:
#extract study termination data
supplementation_data = project.export_records(forms=['maternal_supplementation'])

#convert data to Dataframe.
supp_data=pd.DataFrame(supplementation_data)

In [5]:
# Specify the fields you want to extract base on logics
df_suppData=supp_data[['con_participantid_q1','redcap_event_name','redcap_repeat_instrument','redcap_repeat_instance','ms_dvisit_q1','ms_completed_by']]
df_suppData=pd.DataFrame(df_suppData)

#st_date as a date data type
df_suppData['ms_dvisit_q1'] = pd.to_datetime(df_suppData['ms_dvisit_q1'], errors='coerce')


df_supp = df_suppData[
    (df_suppData['redcap_event_name'] == 'daily_supplementat_arm_1') &
    (df_suppData['con_participantid_q1'].eq(df_suppData['con_participantid_q1'].shift(-1))) &
    ((df_suppData['ms_dvisit_q1'].shift(-1) - df_suppData['ms_dvisit_q1']).dt.days != 1)
]


In [6]:
#Daily supplementation date diffference
df_supp = pd.DataFrame({
    'ID': df_suppData['con_participantid_q1'].shift(-1)+"-"+(df_supp['redcap_repeat_instance'] + 1).astype(str)+"-"+(pd.to_datetime(df_suppData['ms_dvisit_q1'].shift(-1))).astype(str),
    'con_participantid_q1': df_suppData['con_participantid_q1'].shift(-1),
    'Instance number': df_suppData['redcap_repeat_instance'].shift(-1),
    'Supplementation date': pd.to_datetime(df_suppData['ms_dvisit_q1'].shift(-1)),
    'Instrument': 'Maternal Supplementation',
    'Completed by' : df_suppData['ms_completed_by'].shift(-1),
    'Description':  'This date difference between instance ' + df_supp['redcap_repeat_instance'].astype(str) + ' and instance ' + (df_supp['redcap_repeat_instance'] + 1).astype(str) + ' is not equals to 1.'

})

In [7]:
## EXCLUDE ALL RESOLVED QUERIES
#Read from the ones that are resolved: "RESOLVED_QUERIES.csv"
df_resolvedQ = pd.read_csv('RESOLVED_QUERIES.csv')

# Merge the dataframes on 'ID'
con_merged_df = pd.merge(df_supp, df_resolvedQ, on= 'ID', how='inner')

# Filter out the common ID
df_supp= df_supp[~df_supp['ID'].isin(con_merged_df['ID'])]

In [8]:
#Final Infant Query data frame
df_supp = pd.DataFrame({
    'ID': df_supp['ID'],
    'con_participantid_q1': df_supp['con_participantid_q1'],
    'Instance number': df_supp['Instance number'],
    'Supplementation date': df_supp['Supplementation date'],
    'Instrument': df_supp['Instrument'],
    'Completed by' : df_supp['Completed by'],
    'Description':  df_supp['Description']

})

In [9]:
df_supp=df_supp[
    (df_supp['Description'].notnull())
]

In [10]:
#Save the filtered DataFrame to a CSV file
df_supp.to_csv('SupplementationQuery.csv', index=False)